# WatsonxRerank

> WatsonxRerank 是 IBM [watsonx.ai](https://www.ibm.com/products/watsonx-ai) 基础模型的封装器。

该 Notebook 展示如何在检索器中使用 [watsonx 的 rerank 端点](https://cloud.ibm.com/apidocs/watsonx-ai#text-rerank)。这建立在 [ContextualCompressionRetriever](/docs/how_to/contextual_compression) 的思想之上。

## 概览

### 集成详情

| 类 | 包 | [JS 支持](https://js.langchain.com/docs/integrations/document_compressors/ibm/) | 包下载量 | 包最新版本 |
| :--- | :--- | :---: | :---: | :---: |
| [WatsonxRerank](https://python.langchain.com/api_reference/ibm/rerank/langchain_ibm.rerank.WatsonxRerank.html) | [langchain-ibm](https://python.langchain.com/api_reference/ibm/index.html) | ✅ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-ibm?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-ibm?style=flat-square&label=%20) |

## 设置

要访问 IBM watsonx.ai 模型，您需要创建一个 IBM watsonx.ai 账户，获取 API 密钥，并安装 `langchain-ibm` 集成包。

### 凭证

下面的单元格定义了使用 watsonx 基础模型推理所需的凭证。

**操作：** 提供 IBM Cloud 用户 API 密钥。有关详细信息，请参阅
[管理用户 API 密钥](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui)。

In [ ]:
import os
from getpass import getpass

watsonx_api_key = getpass()
os.environ["WATSONX_APIKEY"] = watsonx_api_key

此外，您还可以将额外的 secrets 作为环境变量传递。

In [ ]:
import os

os.environ["WATSONX_URL"] = "your service instance url"
os.environ["WATSONX_TOKEN"] = "your token for accessing the CPD cluster"
os.environ["WATSONX_PASSWORD"] = "your password for accessing the CPD cluster"
os.environ["WATSONX_USERNAME"] = "your username for accessing the CPD cluster"
os.environ["WATSONX_INSTANCE_ID"] = "your instance_id for accessing the CPD cluster"

### 安装

LangChain IBM 集成位于 `langchain-ibm` 包中：

In [1]:
!pip install -qU langchain-ibm
!pip install -qU langchain-community
!pip install -qU langchain_text_splitters

为了实验目的，请同时安装 `faiss` 或 `faiss-cpu` 包：

In [ ]:
!pip install --upgrade --quiet  faiss

# OR  (depending on Python version)

!pip install --upgrade --quiet  faiss-cpu

用于打印文档的辅助函数

In [8]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

## 实例化

### 设置基础向量存储检索器
让我们开始初始化一个简单的向量存储检索器，并存储 2023 年国情咨文（分块）。我们可以将检索器设置为检索大量（20 个）文档。

初始化 `WatsonxEmbeddings`。有关更多详细信息，请参阅 [WatsonxEmbeddings](/docs/integrations/text_embedding/ibm_watsonx)。

**注意**：

- 为 API 调用提供上下文时，必须添加 `project_id` 或 `space_id`。有关更多信息，请参阅[文档](https://www.ibm.com/docs/en/watsonx-as-a-service?topic=projects)。
- 根据您提供的服务实例的区域，使用此处描述的 URL 之一[此处](https://ibm.github.io/watsonx-ai-python-sdk/setup_cloud.html#authentication)。

在此示例中，我们将使用 `project_id` 和 Dallas URL。

您需要指定将用于嵌入的 `model_id`。所有可用模型都可以在[文档](https://ibm.github.io/watsonx-ai-python-sdk/fm_embeddings.html#EmbeddingModels)中找到。

In [ ]:
from langchain_ibm import WatsonxEmbeddings

wx_embeddings = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="PASTE YOUR PROJECT_ID HERE",
)

In [22]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = TextLoader("../../how_to/state_of_the_union.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts, wx_embeddings).as_retriever(
    search_kwargs={"k": 20}
)

query = "What did the president say about Ketanji Brown Jackson"
docs = retriever.invoke(query)
pretty_print_docs(docs[:5])  # Printing the first 5 documents

Document 1:

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
----------------------------------------------------------------------------------------------------
Document 2:

I spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves. 

I’ve worked on these issues a long time. 

I know what works: Investing in crime prevention and community police officers who’ll walk the beat, who’ll know the neighborhood, and who can restore trust and safety. 

So let’s not abandon our streets. Or choose between safety and equal justice.
--------------------------------------------------------------

## 用法

### 使用 WatsonxRerank 进行重排序
现在我们用 `ContextualCompressionRetriever` 来包装我们的基础检索器。我们将添加一个 `WatsonxRerank`，它使用 watsonx rerank 端点来重排序返回的结果。
请注意，在 WatsonxRerank 中指定模型名称是强制性的！

In [ ]:
from langchain_ibm import WatsonxRerank

wx_rerank = WatsonxRerank(
    model_id="cross-encoder/ms-marco-minilm-l-12-v2",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="PASTE YOUR PROJECT_ID HERE",
)

In [23]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever

compression_retriever = ContextualCompressionRetriever(
    base_compressor=wx_rerank, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
pretty_print_docs(compressed_docs[:5])  # Printing the first 5 compressed documents

Document 1:

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
----------------------------------------------------------------------------------------------------
Document 2:

As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. 

While it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice.
----------------------------------------------------------------------------------------------------
Document 3:

To al

## 在链中使用

当然，你可以在 QA 管道中使用这个检索器。

初始化 `ChatWatsonx`。更多详情请参阅 [ChatWatsonx](/docs/integrations/chat/ibm_watsonx)。

In [ ]:
from langchain_ibm import ChatWatsonx

wx_chat = ChatWatsonx(
    model_id="meta-llama/llama-3-1-70b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="PASTE YOUR PROJECT_ID HERE",
)

In [19]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=wx_chat, retriever=compression_retriever)

In [20]:
chain.invoke(query)

{'query': 'What did the president say about Ketanji Brown Jackson',
 'result': 'The President said that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and described her as "one of our nation\'s top legal minds" who will continue Justice Breyer\'s legacy of excellence.'}

## API 参考

有关 `WatsonxRerank` 所有功能和配置的详细文档，请前往 [API 参考](https://python.langchain.com/api_reference/ibm/chat_models/langchain_ibm.rerank.WatsonxRerank.html)。